# Adjacency Matrix (AM) as HRT (Hash Relational Tensor)

>hash-addressable, dynamically-growing, PyTorch-only implementation

```math
r(hᵢ, hⱼ) ← v  (\text{ overwrite if exists, insert if new})
```

for an arbitrary number of tokens whose only identifier is the hash value h = hash(tᵢ).


No Python loops touch the tensor itself; all lookups and updates are O(1) and executed with one tensor op.

## Data layout

We keep a single 2-D tensor R of shape (capacity, capacity)
row/col indices are internal integer ids (0,1,2,…) that we map to/from the hash values with two dictionaries:

```text
hash → idx   : self.h2i
idx  → hash  : self.i2h
```

capacity grows automatically in powers of two (doubling when >75 % full) and the old contents are copied into the new larger tensor with a single index_copy.

In [1]:
import torch
import os

# Check available GPUs
# print("Available GPUs:")
for i in range(torch.cuda.device_count()):
    props = torch.cuda.get_device_properties(i)
#     print(f"  GPU {i}: {props.name}")
#     print(f"    Compute Capability: sm_{props.major}{props.minor}")
#     print(f"    Memory: {props.total_memory / 1024**3:.2f} GB")

# Select RTX 3060 (adjust index based on output above)
# Option A: Hide Quadro, only show RTX
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Assuming RTX is at index 1
DEVICE = torch.device("cuda:0")  # Now index 0 refers to the RTX

# Option B: Explicitly select by index (if you don't set CUDA_VISIBLE_DEVICES)
DEVICE = torch.device("cuda:1")  # Direct access to RTX at original index 1

# print(f"\nUsing device: {DEVICE}")
# if DEVICE.type == "cuda":
#     print(f"Device name: {torch.cuda.get_device_name(DEVICE)}")
#     print(f"Memory: {torch.cuda.get_device_properties(DEVICE).total_memory / 1024**3:.2f} GB")

/home/alexmy/SGS/SGS_lib/hllset-swarm-kimi/.venv/lib64/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU1 Quadro M1200 which is of cuda capability 5.0.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  warnings.warn(
/home/alexmy/SGS/SGS_lib/hllset-swarm-kimi/.venv/lib64/python3.12/site-packages/torch/cuda/__init__.py:304: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  warnings.warn(matched_cuda_warn.format(matched_arches))
/home/alexmy/SGS/SGS_lib/hllset-swarm-kimi/.venv/lib64/python3.12/site-packages/torch/cuda/__init__.py:326: UserWarning: 
Quadro M1200 with CUDA capability sm_50 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the Quadro M1200 GPU with P

## Demo

### Complexity

Time: O(|keep|²) – once.
Memory: the old matrix is discarded immediately; only the pruned one remains.
All subsequent operations (update, get_history, project, …) run on the smaller matrix.

In [10]:
import json
from pathlib import Path
from src.hllset_swarm.swarm_hrt import SwarmHRT
from src.hllset_swarm.constants import HASH_FUNC
from src.hllset_swarm.ingest import ingest_stream
# from src.hllset_swarm import SwarmHRT, ingest_stream, HASH_FUNC, P_BITS, SHARED_SEED

def demo():
    hrt = SwarmHRT(max_edges=20_000)
    corpus = ["人工智能引领未来", "机器学习改变世界", "深度学习驱动创新"]
    commits = []
    # belief = torch.zeros(len(hrt.h2i), device="cuda", dtype=torch.float32)
    for belief_vec, commit in ingest_stream(corpus, hrt, swarm_iters_per_chunk=3):
        commits.append(commit)

    belief = torch.zeros(len(hrt.h2i), device="cuda", dtype=torch.float32)

    # ---- guided finish ----
    dest_hashes = [HASH_FUNC(c) for c in ["未", "来", "世", "界"]]
    p_star = torch.zeros(len(hrt.h2i), device="cuda", dtype=torch.float32)
    for h in dest_hashes:
        if h in hrt.h2i:
            p_star[hrt.h2i[h]] = 1.0
    p_star = p_star / p_star.sum()
    for step in range(5):
        belief = hrt.guided_step(belief, p_star)
        if torch.norm(belief - p_star, 1) < 1e-3:
            break
    top = torch.topk(belief, 18).indices
    # tokens = [hrt.i2h[i.item()] for i in top]
    tokens = [hrt.h2token.get(hrt.i2h[i.item()], f"<unk_{i.item()}>") for i in top]

    print("Destination tokens:", "".join(tokens))
    Path("kimi_git_log.json").write_text(json.dumps(commits, indent=2))

In [11]:
demo()


Destination tokens: 领未领未来工智工智能人工人人工智工能引领能引智能引能引领引引领未领智能智


## Complexity

- update/get: O(1) average time, one tensor write.
- memory: O(N²) where N = number of distinct hashes (only the actually used N×N block is returned by get_dense()).
- growth: amortised O(1); when capacity doubles we pay one index_copy of the existing N×N block.

## Extensions

1. Symmetric relation: mirror each write R[j,i] = R[i,j].
2. Directed vs undirected: store only upper triangle and use a triu_indices view.
3. Mini-batch updates: vectorise with index_put_ and two integer tensors rows, cols.